In [28]:
import pandas as pd
from vega_datasets import data
import draco
from draco import dict_to_facts, schema_from_dataframe


In [1]:
import load_dotenv
import os

In [2]:
load_dotenv.load_dotenv()

True

In [7]:
example_data_schemas = '''
{'number_rows': 1461,
 'field': [{'name': 'date',
   'type': 'datetime',
   'unique': 1461,
   'entropy': 7287},
  {'name': 'precipitation',
   'type': 'number',
   'unique': 111,
   'entropy': 2422,
   'min': 0,
   'max': 55,
   'std': 6},
  {'name': 'temp_max',
   'type': 'number',
   'unique': 67,
   'entropy': 3934,
   'min': -1,
   'max': 35,
   'std': 7},
  {'name': 'temp_min',
   'type': 'number',
   'unique': 55,
   'entropy': 3596,
   'min': -7,
   'max': 18,
   'std': 5},
  {'name': 'wind',
   'type': 'number',
   'unique': 79,
   'entropy': 3950,
   'min': 0,
   'max': 9,
   'std': 1},
  {'name': 'weather',
   'type': 'string',
   'unique': 5,
   'entropy': 1201,
   'freq': np.int64(714)}]}
'''

tick_plot = '''
# Tick Plot
{
  'number_rows': 1461,
  'field': [
    {
      'name': 'temp_max',
      'type': 'number',
      '__id__': 'temp_max'
    }
  ],
  'view': [
    {
      'mark': [
        {
          'type': 'tick',
          'encoding': [
            {
              'channel': 'x',
              'field': 'temp_max'
            }
          ]
        }
      ],
      'scale': [
        {
          'channel': 'x',
          'type': 'linear'
        }
      ]
    }
  ]
}
'''

tick_plot_log = '''
# Tick Plot with a Log Scale
{
  'number_rows': 1456,
  'field': [
    {
      'name': 'temp_max',
      'type': 'number',
      '__id__': 'temp_max'
    }
  ],
  'view': [
    {
      'mark': [
        {
          'type': 'tick',
          'encoding': [
            {
              'channel': 'x',
              'field': 'temp_max'
            }
          ]
        }
      ],
      'scale': [
        {
          'channel': 'x',
          'type': 'log'
        }
      ]
    }
  ]
}
'''

bar_chart = '''
# Bar Chart
{
  'number_rows': 1461,
  'field': [
    {
      'name': 'temp_max',
      'type': 'number',
      '__id__': 'temp_max'
    },
    {
      'name': 'weather',
      'type': 'string',
      '__id__': 'weather'
    }
  ],
  'view': [
    {
      'coordinates': 'cartesian',
      'mark': [
        {
          'type': 'bar',
          'encoding': [
            {
              'channel': 'x',
              'field': 'weather'
            },
            {
              'channel': 'y',
              'field': 'temp_max',
              'aggregate': 'mean'
            }
          ]
        }
      ],
      'scale': [
        {
          'channel': 'x',
          'type': 'ordinal'
        },
        {
          'channel': 'y',
          'type': 'linear',
          'zero': 'true'
        }
      ]
    }
  ]
}
'''

radial_chart = '''
# Radial Chart
{
  'number_rows': 1461,
  'field': [
    {
      'name': 'temp_max',
      'type': 'number',
      '__id__': 'temp_max'
    },
    {
      'name': 'weather',
      'type': 'string',
      '__id__': 'weather'
    }
  ],
  'view': [
    {
      'coordinates': 'polar',
      'mark': [
        {
          'type': 'bar',
          'encoding': [
            {
              'channel': 'x',
              'field': 'weather'
            },
            {
              'channel': 'y',
              'field': 'temp_max',
              'aggregate': 'mean'
            }
          ]
        }
      ],
      'scale': [
        {
          'channel': 'x',
          'type': 'ordinal'
        },
        {
          'channel': 'y',
          'type': 'linear',
          'zero': 'true'
        }
      ]
    }
  ]
}
'''

histogram = '''
{
  'number_rows': 1461,
  'field': [
    {
      'name': 'weather',
      'type': 'string',
      '__id__': 'weather'
    }
  ],
  'view': [
    {
      'mark': [
        {
          'type': 'bar',
          'encoding': [
            {
              'channel': 'x',
              'field': 'weather'
            },
            {
              'channel': 'y',
              'aggregate': 'count'
            }
          ]
        }
      ],
      'scale': [
        {
          'channel': 'x',
          'type': 'ordinal'
        },
        {
          'channel': 'y',
          'type': 'linear',
          'zero': 'true'
        }
      ]
    }
  ]
}
'''

binned_histogram = '''
{
  'number_rows': 1461,
  'field': [
    {
      'name': 'temp_max',
      'type': 'number',
      '__id__': 'temp_max'
    }
  ],
  'view': [
    {
      'mark': [
        {
          'type': 'bar',
          'encoding': [
            {
              'channel': 'x',
              'field': 'temp_max',
              'binning': 10
            },
            {
              'channel': 'y',
              'aggregate': 'count'
            }
          ]
        }
      ],
      'scale': [
        {
          'channel': 'x',
          'type': 'linear'
        },
        {
          'channel': 'y',
          'type': 'linear',
          'zero': 'true'
        }
      ]
    }
  ]
}
'''


scatter_plot = '''
{
  'number_rows': 1461,
  'field': [
    {
      'name': 'temp_max',
      'type': 'number',
      '__id__': 'temp_max'
    },
    {
      'name': 'wind',
      'type': 'number',
      '__id__': 'wind'
    }
  ],
  'view': [
    {
      'mark': [
        {
          'type': 'point',
          'encoding': [
            {
              'channel': 'x',
              'field': 'temp_max'
            },
            {
              'channel': 'y',
              'field': 'wind'
            }
          ]
        }
      ],
      'scale': [
        {
          'channel': 'x',
          'type': 'linear'
        },
        {
          'channel': 'y',
          'type': 'linear'
        }
      ]
    }
  ]
}
'''

scatter_plot_color = '''
{
  'number_rows': 1461,
  'field': [
    {
      'name': 'temp_max',
      'type': 'number',
      '__id__': 'temp_max'
    },
    {
      'name': 'wind',
      'type': 'number',
      '__id__': 'wind'
    },
    {
      'name': 'weather',
      'type': 'string',
      '__id__': 'weather'
    }
  ],
  'view': [
    {
      'mark': [
        {
          'type': 'point',
          'encoding': [
            {
              'channel': 'x',
              'field': 'temp_max'
            },
            {
              'channel': 'y',
              'field': 'wind'
            },
            {
              'channel': 'color',
              'field': 'weather'
            }
          ]
        }
      ],
      'scale': [
        {
          'channel': 'x',
          'type': 'linear'
        },
        {
          'channel': 'y',
          'type': 'linear'
        },
        {
          'channel': 'color',
          'type': 'categorical'
        }
      ]
    }
  ]
}
'''

bubble_chart = '''
{
  'number_rows': 1461,
  'field': [
    {
      'name': 'temp_max',
      'type': 'number',
      '__id__': 'temp_max'
    },
    {
      'name': 'wind',
      'type': 'number',
      '__id__': 'wind'
    },
    {
      'name': 'precipitation',
      'type': 'number',
      '__id__': 'precipitation'
    }
  ],
  'view': [
    {
      'mark': [
        {
          'type': 'point',
          'encoding': [
            {
              'channel': 'x',
              'field': 'temp_max'
            },
            {
              'channel': 'y',
              'field': 'wind'
            },
            {
              'channel': 'size',
              'field': 'precipitation'
            }
          ]
        }
      ],
      'scale': [
        {
          'channel': 'x',
          'type': 'linear'
        },
        {
          'channel': 'y',
          'type': 'linear'
        },
        {
          'channel': 'size',
          'type': 'linear'
        }
      ]
    }
  ]
}
'''

pie_chart = '''
{
  'number_rows': 1461,
  'field': [
    {
      'name': 'temp_max',
      'type': 'number',
      '__id__': 'temp_max'
    },
    {
      'name': 'weather',
      'type': 'string',
      '__id__': 'weather'
    }
  ],
  'view': [
    {
      'coordinates': 'polar',
      'mark': [
        {
          'type': 'bar',
          'encoding': [
            {
              'channel': 'y',
              'aggregate': 'count',
              'stack': 'zero'
            },
            {
              'channel': 'color',
              'field': 'weather'
            }
          ]
        }
      ],
      'scale': [
        {
          'channel': 'y',
          'type': 'linear',
          'zero': 'true'
        },
        {
          'channel': 'color',
          'type': 'categorical'
        }
      ]
    }
  ]
}
'''

stacked_bar_chart = '''
{
  'number_rows': 1461,
  'field': [
    {
      'name': 'temp_max',
      'type': 'number',
      '__id__': 'temp_max'
    },
    {
      'name': 'weather',
      'type': 'string',
      '__id__': 'weather'
    }
  ],
  'view': [
    {
      'mark': [
        {
          'type': 'bar',
          'encoding': [
            {
              'channel': 'x',
              'field': 'temp_max',
              'binning': 10
            },
            {
              'channel': 'y',
              'aggregate': 'count',
              'stack': 'zero'
            },
            {
              'channel': 'color',
              'field': 'weather'
            }
          ]
        }
      ],
      'scale': [
        {
          'channel': 'x',
          'type': 'linear'
        },
        {
          'channel': 'y',
          'type': 'linear',
          'zero': 'true'
        },
        {
          'channel': 'color',
          'type': 'categorical'
        }
      ]
    }
  ]
}
'''

normolized_stacked_bar_chart = '''
{
  'number_rows': 1461,
  'field': [
    {
      'name': 'temp_max',
      'type': 'number',
      '__id__': 'temp_max'
    },
    {
      'name': 'weather',
      'type': 'string',
      '__id__': 'weather'
    }
  ],
  'view': [
    {
      'mark': [
        {
          'type': 'bar',
          'encoding': [
            {
              'channel': 'x',
              'aggregate': 'count',
              'stack': 'normalize'
            },
            {
              'channel': 'y',
              'field': 'temp_max',
              'binning': 10
            },
            {
              'channel': 'color',
              'field': 'weather'
            }
          ]
        }
      ],
      'scale': [
        {
          'channel': 'x',
          'type': 'linear',
          'zero': 'true'
        },
        {
          'channel': 'y',
          'type': 'linear'
        },
        {
          'channel': 'color',
          'type': 'categorical'
        }
      ]
    }
  ]
}
'''

bar_with_tick = '''
{
  'number_rows': 1461,
  'field': [
    {
      'name': 'temp_max',
      'type': 'number',
      '__id__': 'temp_max'
    }
  ],
  'view': [
    {
      'mark': [
        {
          'type': 'bar',
          'encoding': [
            {
              'channel': 'x',
              'aggregate': 'mean',
              'field': 'temp_max'
            }
          ]
        },
        {
          'type': 'tick',
          'encoding': [
            {
              'channel': 'x',
              'field': 'temp_max'
            }
          ]
        }
      ],
      'scale': [
        {
          'channel': 'x',
          'type': 'linear',
          "zero": "true"
        }
      ]
    }
  ]
}
'''

facet_scatter_plot_column = '''
{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    },
    {
      "name": "wind",
      "type": "number",
      "__id__": "wind"
    },
    {
      "name": "weather",
      "type": "string",
      "__id__": "weather"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "point",
          "encoding": [
            {
              "channel": "x",
              "field": "temp_max"
            },
            {
              "channel": "y",
              "field": "wind"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "linear"
        },
        {
          "channel": "y",
          "type": "linear"
        }
      ],
      "facet": [
        {
          "channel": "col",
          "field": "weather"
        }
      ]
    }
  ]
}
'''


facet_scatter_plot_binned = '''
{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    },
    {
      "name": "wind",
      "type": "number",
      "__id__": "wind"
    },
    {
      "name": "weather",
      "type": "string",
      "__id__": "weather"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "point",
          "encoding": [
            {
              "channel": "x",
              "field": "weather"
            },
            {
              "channel": "y",
              "field": "wind"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "ordinal"
        },
        {
          "channel": "y",
          "type": "linear"
        }
      ],
      "facet": [
        {
          "channel": "col",
          "field": "temp_max",
          "binning": 10
        }
      ]
    }
  ]
}
'''

tick_plot_histogram = '''
{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    },
    {
      "name": "weather",
      "type": "string",
      "__id__": "weather"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "tick",
          "encoding": [
            {
              "channel": "y",
              "field": "temp_max"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "y",
          "type": "linear",
          "zero": "true"
        }
      ]
    },
    {
      "mark": [
        {
          "type": "bar",
          "encoding": [
            {
              "channel": "x",
              "field": "weather"
            },
            {
              "channel": "y",
              "aggregate": "count"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "ordinal"
        },
        {
          "channel": "y",
          "type": "linear",
          "zero": "true"
        }
      ]
    }
  ]
}
'''

tick_plot_histogram_y_scale = '''
{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    },
    {
      "name": "weather",
      "type": "string",
      "__id__": "weather"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "tick",
          "encoding": [
            {
              "channel": "y",
              "field": "temp_max"
            }
          ]
        }
      ]
    },
    {
      "mark": [
        {
          "type": "bar",
          "encoding": [
            {
              "channel": "y",
              "field": "temp_max",
              "aggregate": "mean"
            },
            {
              "channel": "x",
              "field": "weather"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "ordinal"
        }
      ]
    }
  ],
  "scale": [
    {
      "channel": "y",
      "type": "linear",
      "zero": "true"
    }
  ]
}
'''

example_plot_schemas = [
    tick_plot,
    tick_plot_log,
    bar_chart,
    radial_chart,
    histogram,
    binned_histogram,
    scatter_plot,
    scatter_plot_color,
    bubble_chart,
    pie_chart,
    stacked_bar_chart,
    normolized_stacked_bar_chart,
    bar_with_tick,
    facet_scatter_plot_column,
    facet_scatter_plot_binned,
    tick_plot_histogram,
    tick_plot_histogram_y_scale,
]

In [11]:
example_plot_schemas_str = '\n'.join(example_plot_schemas)

In [12]:
systme_prompt = f'''
You are a very good data scientist and you have been tasked with creating a visualization for the user data.
Your job is to generate plot schemas for provided data schema. 
Here is an example of the data schema:
{example_data_schemas}

Here are some example plot schemas:
{example_plot_schemas_str}

Now the user will provide his data schema and you have to generate a plot schema for him.
'''

In [53]:
# Loading data
# df: pd.DataFrame = data.gapminder_health_income()
# df: pd.DataFrame = data.birdstrikes()
df = data.population_engineers_hurricanes()

# df['Cost_Total'] = df['Cost__Total_$']
# df.drop(columns=['Cost__Total_$'], inplace=True)
# df = df.iloc[:4000]

schema = schema_from_dataframe(df)
schema
df.head()

,state,id,population,engineers,hurricanes
0,Alabama,1,4863300,0.003422,22
1,Alaska,2,741894,0.001591,0
2,Arizona,4,6931071,0.004774,0
3,Arkansas,5,2988248,0.002440,0
4,California,6,39250017,0.007126,0


In [59]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[
        {
            'role': 'system', 
            'content': example_plot_schemas_str
        },
        {
            'role': 'user',
            'content': str(schema) + 'Now generate a plot schema for this data schema. Provide only a json schema, no explanations or comments.'
        }
    ]
)

In [60]:
from json import loads

from IPython.display import display

from draco import Draco, dict_to_facts, dict_union
from draco.renderer import AltairRenderer

d = Draco()
renderer = AltairRenderer(concat_mode="hconcat")

In [61]:
import re

json_str = re.search(r'\{.*\}', completion.choices[0].message.content, re.DOTALL).group()
plot_schema = loads(json_str)

In [62]:
plot_schema

{'number_rows': 52,
 'field': [{'name': 'state', 'type': 'string', '__id__': 'state'},
  {'name': 'population', 'type': 'number', '__id__': 'population'},
  {'name': 'hurricanes', 'type': 'number', '__id__': 'hurricanes'}],
 'view': [{'mark': [{'type': 'bar',
     'encoding': [{'channel': 'x', 'field': 'state'},
      {'channel': 'y', 'field': 'population', 'aggregate': 'sum'}]}],
   'scale': [{'channel': 'x', 'type': 'ordinal'},
    {'channel': 'y', 'type': 'linear', 'zero': 'true'}]},
  {'mark': [{'type': 'bar',
     'encoding': [{'channel': 'x', 'field': 'state'},
      {'channel': 'y', 'field': 'hurricanes', 'aggregate': 'sum'}]}],
   'scale': [{'channel': 'x', 'type': 'ordinal'},
    {'channel': 'y', 'type': 'linear', 'zero': 'true'}]}]}

In [63]:
prog = dict_to_facts(plot_schema)

if not d.check_spec(prog):
    print("\n".join(prog))
    print(d.get_violations(prog))
    assert False, "Invalid spec"

# Display the rendered VL chart and the ASP
chart = renderer.render(plot_schema, df)
display(chart)
display(prog)

alt.HConcatChart(...)

['attribute(number_rows,root,52).',
 'entity(field,root,state).',
 'attribute((field,name),state,state).',
 'attribute((field,type),state,string).',
 'entity(field,root,population).',
 'attribute((field,name),population,population).',
 'attribute((field,type),population,number).',
 'entity(field,root,hurricanes).',
 'attribute((field,name),hurricanes,hurricanes).',
 'attribute((field,type),hurricanes,number).',
 'entity(view,root,0).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,bar).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,x).',
 'attribute((encoding,field),2,state).',
 'entity(encoding,1,3).',
 'attribute((encoding,channel),3,y).',
 'attribute((encoding,field),3,population).',
 'attribute((encoding,aggregate),3,sum).',
 'entity(scale,0,4).',
 'attribute((scale,channel),4,x).',
 'attribute((scale,type),4,ordinal).',
 'entity(scale,0,5).',
 'attribute((scale,channel),5,y).',
 'attribute((scale,type),5,linear).',
 'attribute((scale,zero),5,true).',
 'entity(view

In [7]:
from pprint import pprint

from draco import Draco, answer_set_to_dict

d = Draco()

In [8]:
from IPython.display import display
from vega_datasets import data

from draco.renderer import AltairRenderer

# Setting up renderer and demo data
renderer = AltairRenderer()
weather_data = data.seattle_weather()

In [9]:
import importlib.resources as pkg_resources

import draco.asp.examples as examples

hist_spec = pkg_resources.read_text(examples, 'histogram.lp')

print('INPUT:')
print(hist_spec)

print('OUTPUT:')
model = next(d.complete_spec(hist_spec))
spec = answer_set_to_dict(model.answer_set)
pprint(spec)
display(renderer.render(spec=spec, data=weather_data))

print('VIOLATED PREFERENCES:')
pprint(d.count_preferences(str(model)))

INPUT:
attribute(number_rows,root,100).

entity(field,root,(f,0)).
attribute((field,name),(f,0),temp_max).
attribute((field,type),(f,0),number).
attribute((field,unique),(f,0),100).

entity(view,root,(v,0)).

entity(mark,(v,0),(m,0)).
entity(encoding,(m,0),(e,0)).
attribute((encoding,field),(e,0),temp_max).
attribute((encoding,binning),(e,0),10).

#show entity/3.
#show attribute/3.

OUTPUT:
{'field': [{'name': 'temp_max', 'type': 'number', 'unique': 100}],
 'number_rows': 100,
 'task': 'summary',
 'view': [{'coordinates': 'cartesian',
           'mark': [{'encoding': [{'binning': 10,
                                   'channel': 'x',
                                   'field': 'temp_max'},
                                  {'aggregate': 'count', 'channel': 'y'}],
                     'type': 'bar'}],
           'scale': [{'channel': 'x', 'type': 'linear'},
                     {'channel': 'y', 'type': 'linear', 'zero': 'true'}]}]}


/tmp/ipykernel_3540/2067865367.py:5: DeprecationWarning: read_text is deprecated. Use files() instead. Refer to https://importlib-resources.readthedocs.io/en/latest/using.html#migrating-from-legacy for migration advice.
  hist_spec = pkg_resources.read_text(examples, "histogram.lp")


alt.Chart(...)

VIOLATED PREFERENCES:
defaultdict(<class 'int'>,
            {'aggregate': 1,
             'aggregate_count': 1,
             'bin': 1,
             'c_d_no_overlap_bar': 1,
             'cartesian_coordinate': 1,
             'encoding': 2,
             'encoding_field': 1,
             'linear_scale': 2,
             'linear_x': 1,
             'linear_y': 1,
             'summary_bar': 1})


In [5]:
import json

from IPython.display import display

from draco import Draco, dict_to_facts, dict_union
from draco.renderer import AltairRenderer

d = Draco()
renderer = AltairRenderer(concat_mode='hconcat')


def show(*args, df: pd.DataFrame = weather_data):
    spec = dict_union(*args)
    prog = dict_to_facts(spec)
    if not d.check_spec(prog):
        print('\n'.join(prog))
        print(d.get_violations(prog))
        assert False, 'Invalid spec'

    # Display the rendered VL chart and the ASP
    chart = renderer.render(spec, df)
    print(json.dumps(spec, indent=2))
    display(chart)
    display(prog)

In [6]:
def data(fields: list[str], df: pd.DataFrame = weather_data) -> dict:
    number_rows, _ = df.shape
    return {
        'number_rows': number_rows,
        'field': [
            x
            for x in [
                {'name': 'temp_max', 'type': 'number', '__id__': 'temp_max'},
                {'name': 'wind', 'type': 'number', '__id__': 'wind'},
                {'name': 'precipitation', 'type': 'number', '__id__': 'precipitation'},
                {'name': 'weather', 'type': 'string', '__id__': 'weather'},
            ]
            if x['name'] in fields
        ],
    }

In [6]:
show(
    data(['temp_max']),
    {
        'view': [
            {
                'mark': [
                    {
                        'type': 'tick',
                        'encoding': [{'channel': 'x', 'field': 'temp_max'}],
                    }
                ],
                'scale': [{'channel': 'x', 'type': 'linear'}],
            }
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "tick",
          "encoding": [
            {
              "channel": "x",
              "field": "temp_max"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "linear"
        }
      ]
    }
  ]
}


alt.Chart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(view,root,0).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,tick).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,x).',
 'attribute((encoding,field),2,temp_max).',
 'entity(scale,0,3).',
 'attribute((scale,channel),3,x).',
 'attribute((scale,type),3,linear).']

In [8]:
show(
    data(['temp_max'], df=weather_data_positive),
    {
        'view': [
            {
                'mark': [
                    {
                        'type': 'tick',
                        'encoding': [{'channel': 'x', 'field': 'temp_max'}],
                    }
                ],
                'scale': [{'channel': 'x', 'type': 'log'}],
            }
        ]
    },
    df=weather_data_positive,
)

{
  "number_rows": 1456,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "tick",
          "encoding": [
            {
              "channel": "x",
              "field": "temp_max"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "log"
        }
      ]
    }
  ]
}


alt.Chart(...)

['attribute(number_rows,root,1456).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(view,root,0).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,tick).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,x).',
 'attribute((encoding,field),2,temp_max).',
 'entity(scale,0,3).',
 'attribute((scale,channel),3,x).',
 'attribute((scale,type),3,log).']

In [10]:
show(
    data(['weather', 'temp_max']),
    {
        'view': [
            {
                'coordinates': 'cartesian',
                'mark': [
                    {
                        'type': 'bar',
                        'encoding': [
                            {'channel': 'x', 'field': 'weather'},
                            {
                                'channel': 'y',
                                'field': 'temp_max',
                                'aggregate': 'mean',
                            },
                        ],
                    }
                ],
                'scale': [
                    {'channel': 'x', 'type': 'ordinal'},
                    {'channel': 'y', 'type': 'linear', 'zero': 'true'},
                ],
            }
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    },
    {
      "name": "weather",
      "type": "string",
      "__id__": "weather"
    }
  ],
  "view": [
    {
      "coordinates": "cartesian",
      "mark": [
        {
          "type": "bar",
          "encoding": [
            {
              "channel": "x",
              "field": "weather"
            },
            {
              "channel": "y",
              "field": "temp_max",
              "aggregate": "mean"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "ordinal"
        },
        {
          "channel": "y",
          "type": "linear",
          "zero": "true"
        }
      ]
    }
  ]
}


alt.Chart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(field,root,weather).',
 'attribute((field,name),weather,weather).',
 'attribute((field,type),weather,string).',
 'entity(view,root,0).',
 'attribute((view,coordinates),0,cartesian).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,bar).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,x).',
 'attribute((encoding,field),2,weather).',
 'entity(encoding,1,3).',
 'attribute((encoding,channel),3,y).',
 'attribute((encoding,field),3,temp_max).',
 'attribute((encoding,aggregate),3,mean).',
 'entity(scale,0,4).',
 'attribute((scale,channel),4,x).',
 'attribute((scale,type),4,ordinal).',
 'entity(scale,0,5).',
 'attribute((scale,channel),5,y).',
 'attribute((scale,type),5,linear).',
 'attribute((scale,zero),5,true).']

In [11]:
show(
    data(['weather', 'temp_max']),
    {
        'view': [
            {
                'coordinates': 'polar',
                'mark': [
                    {
                        'type': 'bar',
                        'encoding': [
                            {'channel': 'x', 'field': 'weather'},
                            {
                                'channel': 'y',
                                'field': 'temp_max',
                                'aggregate': 'mean',
                            },
                        ],
                    }
                ],
                'scale': [
                    {'channel': 'x', 'type': 'ordinal'},
                    {'channel': 'y', 'type': 'linear', 'zero': 'true'},
                ],
            }
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    },
    {
      "name": "weather",
      "type": "string",
      "__id__": "weather"
    }
  ],
  "view": [
    {
      "coordinates": "polar",
      "mark": [
        {
          "type": "bar",
          "encoding": [
            {
              "channel": "x",
              "field": "weather"
            },
            {
              "channel": "y",
              "field": "temp_max",
              "aggregate": "mean"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "ordinal"
        },
        {
          "channel": "y",
          "type": "linear",
          "zero": "true"
        }
      ]
    }
  ]
}


alt.LayerChart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(field,root,weather).',
 'attribute((field,name),weather,weather).',
 'attribute((field,type),weather,string).',
 'entity(view,root,0).',
 'attribute((view,coordinates),0,polar).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,bar).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,x).',
 'attribute((encoding,field),2,weather).',
 'entity(encoding,1,3).',
 'attribute((encoding,channel),3,y).',
 'attribute((encoding,field),3,temp_max).',
 'attribute((encoding,aggregate),3,mean).',
 'entity(scale,0,4).',
 'attribute((scale,channel),4,x).',
 'attribute((scale,type),4,ordinal).',
 'entity(scale,0,5).',
 'attribute((scale,channel),5,y).',
 'attribute((scale,type),5,linear).',
 'attribute((scale,zero),5,true).']

In [12]:
show(
    data(['weather']),
    {
        'view': [
            {
                'mark': [
                    {
                        'type': 'bar',
                        'encoding': [
                            {'channel': 'x', 'field': 'weather'},
                            {'channel': 'y', 'aggregate': 'count'},
                        ],
                    }
                ],
                'scale': [
                    {'channel': 'x', 'type': 'ordinal'},
                    {'channel': 'y', 'type': 'linear', 'zero': 'true'},
                ],
            }
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "weather",
      "type": "string",
      "__id__": "weather"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "bar",
          "encoding": [
            {
              "channel": "x",
              "field": "weather"
            },
            {
              "channel": "y",
              "aggregate": "count"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "ordinal"
        },
        {
          "channel": "y",
          "type": "linear",
          "zero": "true"
        }
      ]
    }
  ]
}


alt.Chart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,weather).',
 'attribute((field,name),weather,weather).',
 'attribute((field,type),weather,string).',
 'entity(view,root,0).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,bar).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,x).',
 'attribute((encoding,field),2,weather).',
 'entity(encoding,1,3).',
 'attribute((encoding,channel),3,y).',
 'attribute((encoding,aggregate),3,count).',
 'entity(scale,0,4).',
 'attribute((scale,channel),4,x).',
 'attribute((scale,type),4,ordinal).',
 'entity(scale,0,5).',
 'attribute((scale,channel),5,y).',
 'attribute((scale,type),5,linear).',
 'attribute((scale,zero),5,true).']

In [13]:
show(
    data(['temp_max']),
    {
        'view': [
            {
                'mark': [
                    {
                        'type': 'bar',
                        'encoding': [
                            {'channel': 'x', 'field': 'temp_max', 'binning': 10},
                            {'channel': 'y', 'aggregate': 'count'},
                        ],
                    }
                ],
                'scale': [
                    {
                        'channel': 'x',
                        'type': 'linear',
                    },
                    {'channel': 'y', 'type': 'linear', 'zero': 'true'},
                ],
            }
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "bar",
          "encoding": [
            {
              "channel": "x",
              "field": "temp_max",
              "binning": 10
            },
            {
              "channel": "y",
              "aggregate": "count"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "linear"
        },
        {
          "channel": "y",
          "type": "linear",
          "zero": "true"
        }
      ]
    }
  ]
}


alt.Chart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(view,root,0).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,bar).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,x).',
 'attribute((encoding,field),2,temp_max).',
 'attribute((encoding,binning),2,10).',
 'entity(encoding,1,3).',
 'attribute((encoding,channel),3,y).',
 'attribute((encoding,aggregate),3,count).',
 'entity(scale,0,4).',
 'attribute((scale,channel),4,x).',
 'attribute((scale,type),4,linear).',
 'entity(scale,0,5).',
 'attribute((scale,channel),5,y).',
 'attribute((scale,type),5,linear).',
 'attribute((scale,zero),5,true).']

In [14]:
show(
    data(['temp_max', 'wind']),
    {
        'view': [
            {
                'mark': [
                    {
                        'type': 'point',
                        'encoding': [
                            {'channel': 'x', 'field': 'temp_max'},
                            {'channel': 'y', 'field': 'wind'},
                        ],
                    }
                ],
                'scale': [
                    {'channel': 'x', 'type': 'linear'},
                    {'channel': 'y', 'type': 'linear'},
                ],
            }
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    },
    {
      "name": "wind",
      "type": "number",
      "__id__": "wind"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "point",
          "encoding": [
            {
              "channel": "x",
              "field": "temp_max"
            },
            {
              "channel": "y",
              "field": "wind"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "linear"
        },
        {
          "channel": "y",
          "type": "linear"
        }
      ]
    }
  ]
}


alt.Chart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(field,root,wind).',
 'attribute((field,name),wind,wind).',
 'attribute((field,type),wind,number).',
 'entity(view,root,0).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,point).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,x).',
 'attribute((encoding,field),2,temp_max).',
 'entity(encoding,1,3).',
 'attribute((encoding,channel),3,y).',
 'attribute((encoding,field),3,wind).',
 'entity(scale,0,4).',
 'attribute((scale,channel),4,x).',
 'attribute((scale,type),4,linear).',
 'entity(scale,0,5).',
 'attribute((scale,channel),5,y).',
 'attribute((scale,type),5,linear).']

In [15]:
show(
    data(['temp_max', 'wind', 'weather']),
    {
        'view': [
            {
                'mark': [
                    {
                        'type': 'point',
                        'encoding': [
                            {'channel': 'x', 'field': 'temp_max'},
                            {'channel': 'y', 'field': 'wind'},
                            {'channel': 'color', 'field': 'weather'},
                        ],
                    }
                ],
                'scale': [
                    {'channel': 'x', 'type': 'linear'},
                    {'channel': 'y', 'type': 'linear'},
                    {'channel': 'color', 'type': 'categorical'},
                ],
            }
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    },
    {
      "name": "wind",
      "type": "number",
      "__id__": "wind"
    },
    {
      "name": "weather",
      "type": "string",
      "__id__": "weather"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "point",
          "encoding": [
            {
              "channel": "x",
              "field": "temp_max"
            },
            {
              "channel": "y",
              "field": "wind"
            },
            {
              "channel": "color",
              "field": "weather"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "linear"
        },
        {
          "channel": "y",
          "type": "linear"
        },
        {
          "channel": "color",
          "type": "categorical"
        }
      ]
    }
  ]
}


alt.Chart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(field,root,wind).',
 'attribute((field,name),wind,wind).',
 'attribute((field,type),wind,number).',
 'entity(field,root,weather).',
 'attribute((field,name),weather,weather).',
 'attribute((field,type),weather,string).',
 'entity(view,root,0).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,point).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,x).',
 'attribute((encoding,field),2,temp_max).',
 'entity(encoding,1,3).',
 'attribute((encoding,channel),3,y).',
 'attribute((encoding,field),3,wind).',
 'entity(encoding,1,4).',
 'attribute((encoding,channel),4,color).',
 'attribute((encoding,field),4,weather).',
 'entity(scale,0,5).',
 'attribute((scale,channel),5,x).',
 'attribute((scale,type),5,linear).',
 'entity(scale,0,6).',
 'attribute((scale,channel),6,y).',
 'attribute((scale,type),6,linear).',
 'en

In [16]:
show(
    data(['temp_max', 'wind', 'precipitation']),
    {
        'view': [
            {
                'mark': [
                    {
                        'type': 'point',
                        'encoding': [
                            {'channel': 'x', 'field': 'temp_max'},
                            {'channel': 'y', 'field': 'wind'},
                            {'channel': 'size', 'field': 'precipitation'},
                        ],
                    }
                ],
                'scale': [
                    {'channel': 'x', 'type': 'linear'},
                    {'channel': 'y', 'type': 'linear'},
                    {'channel': 'size', 'type': 'linear'},
                ],
            }
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    },
    {
      "name": "wind",
      "type": "number",
      "__id__": "wind"
    },
    {
      "name": "precipitation",
      "type": "number",
      "__id__": "precipitation"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "point",
          "encoding": [
            {
              "channel": "x",
              "field": "temp_max"
            },
            {
              "channel": "y",
              "field": "wind"
            },
            {
              "channel": "size",
              "field": "precipitation"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "linear"
        },
        {
          "channel": "y",
          "type": "linear"
        },
        {
          "channel": "size",
          "type": "linear"
        }
      ]
    }
  ]
}


alt.Chart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(field,root,wind).',
 'attribute((field,name),wind,wind).',
 'attribute((field,type),wind,number).',
 'entity(field,root,precipitation).',
 'attribute((field,name),precipitation,precipitation).',
 'attribute((field,type),precipitation,number).',
 'entity(view,root,0).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,point).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,x).',
 'attribute((encoding,field),2,temp_max).',
 'entity(encoding,1,3).',
 'attribute((encoding,channel),3,y).',
 'attribute((encoding,field),3,wind).',
 'entity(encoding,1,4).',
 'attribute((encoding,channel),4,size).',
 'attribute((encoding,field),4,precipitation).',
 'entity(scale,0,5).',
 'attribute((scale,channel),5,x).',
 'attribute((scale,type),5,linear).',
 'entity(scale,0,6).',
 'attribute((scale,channel),6,y).',
 'attribute((

In [18]:
show(
    data(['weather', 'temp_max']),
    {
        'view': [
            {
                'coordinates': 'polar',
                'mark': [
                    {
                        'type': 'bar',
                        'encoding': [
                            {
                                'channel': 'y',
                                'aggregate': 'count',
                                'stack': 'zero',
                            },
                            {'channel': 'color', 'field': 'weather'},
                        ],
                    }
                ],
                'scale': [
                    {'channel': 'y', 'type': 'linear', 'zero': 'true'},
                    {'channel': 'color', 'type': 'categorical'},
                ],
            }
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    },
    {
      "name": "weather",
      "type": "string",
      "__id__": "weather"
    }
  ],
  "view": [
    {
      "coordinates": "polar",
      "mark": [
        {
          "type": "bar",
          "encoding": [
            {
              "channel": "y",
              "aggregate": "count",
              "stack": "zero"
            },
            {
              "channel": "color",
              "field": "weather"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "y",
          "type": "linear",
          "zero": "true"
        },
        {
          "channel": "color",
          "type": "categorical"
        }
      ]
    }
  ]
}


alt.Chart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(field,root,weather).',
 'attribute((field,name),weather,weather).',
 'attribute((field,type),weather,string).',
 'entity(view,root,0).',
 'attribute((view,coordinates),0,polar).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,bar).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,y).',
 'attribute((encoding,aggregate),2,count).',
 'attribute((encoding,stack),2,zero).',
 'entity(encoding,1,3).',
 'attribute((encoding,channel),3,color).',
 'attribute((encoding,field),3,weather).',
 'entity(scale,0,4).',
 'attribute((scale,channel),4,y).',
 'attribute((scale,type),4,linear).',
 'attribute((scale,zero),4,true).',
 'entity(scale,0,5).',
 'attribute((scale,channel),5,color).',
 'attribute((scale,type),5,categorical).']

In [19]:
show(
    data(['temp_max', 'weather']),
    {
        'view': [
            {
                'mark': [
                    {
                        'type': 'bar',
                        'encoding': [
                            {'channel': 'x', 'field': 'temp_max', 'binning': 10},
                            {'channel': 'y', 'aggregate': 'count', 'stack': 'zero'},
                            {'channel': 'color', 'field': 'weather'},
                        ],
                    }
                ],
                'scale': [
                    {
                        'channel': 'x',
                        'type': 'linear',
                    },
                    {'channel': 'y', 'type': 'linear', 'zero': 'true'},
                    {'channel': 'color', 'type': 'categorical'},
                ],
            }
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    },
    {
      "name": "weather",
      "type": "string",
      "__id__": "weather"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "bar",
          "encoding": [
            {
              "channel": "x",
              "field": "temp_max",
              "binning": 10
            },
            {
              "channel": "y",
              "aggregate": "count",
              "stack": "zero"
            },
            {
              "channel": "color",
              "field": "weather"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "linear"
        },
        {
          "channel": "y",
          "type": "linear",
          "zero": "true"
        },
        {
          "channel": "color",
          "type": "categorical"
        }
      ]
    }
  ]
}


alt.Chart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(field,root,weather).',
 'attribute((field,name),weather,weather).',
 'attribute((field,type),weather,string).',
 'entity(view,root,0).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,bar).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,x).',
 'attribute((encoding,field),2,temp_max).',
 'attribute((encoding,binning),2,10).',
 'entity(encoding,1,3).',
 'attribute((encoding,channel),3,y).',
 'attribute((encoding,aggregate),3,count).',
 'attribute((encoding,stack),3,zero).',
 'entity(encoding,1,4).',
 'attribute((encoding,channel),4,color).',
 'attribute((encoding,field),4,weather).',
 'entity(scale,0,5).',
 'attribute((scale,channel),5,x).',
 'attribute((scale,type),5,linear).',
 'entity(scale,0,6).',
 'attribute((scale,channel),6,y).',
 'attribute((scale,type),6,linear).',
 'attribute((scale,zero),6,tru

In [20]:
show(
    data(['temp_max', 'weather']),
    {
        'view': [
            {
                'mark': [
                    {
                        'type': 'bar',
                        'encoding': [
                            {
                                'channel': 'x',
                                'aggregate': 'count',
                                'stack': 'normalize',
                            },
                            {'channel': 'y', 'field': 'temp_max', 'binning': 10},
                            {'channel': 'color', 'field': 'weather'},
                        ],
                    }
                ],
                'scale': [
                    {'channel': 'x', 'type': 'linear', 'zero': 'true'},
                    {'channel': 'y', 'type': 'linear'},
                    {'channel': 'color', 'type': 'categorical'},
                ],
            }
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    },
    {
      "name": "weather",
      "type": "string",
      "__id__": "weather"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "bar",
          "encoding": [
            {
              "channel": "x",
              "aggregate": "count",
              "stack": "normalize"
            },
            {
              "channel": "y",
              "field": "temp_max",
              "binning": 10
            },
            {
              "channel": "color",
              "field": "weather"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "linear",
          "zero": "true"
        },
        {
          "channel": "y",
          "type": "linear"
        },
        {
          "channel": "color",
          "type": "categorical"
        }
      ]
    }
  ]
}


alt.Chart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(field,root,weather).',
 'attribute((field,name),weather,weather).',
 'attribute((field,type),weather,string).',
 'entity(view,root,0).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,bar).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,x).',
 'attribute((encoding,aggregate),2,count).',
 'attribute((encoding,stack),2,normalize).',
 'entity(encoding,1,3).',
 'attribute((encoding,channel),3,y).',
 'attribute((encoding,field),3,temp_max).',
 'attribute((encoding,binning),3,10).',
 'entity(encoding,1,4).',
 'attribute((encoding,channel),4,color).',
 'attribute((encoding,field),4,weather).',
 'entity(scale,0,5).',
 'attribute((scale,channel),5,x).',
 'attribute((scale,type),5,linear).',
 'attribute((scale,zero),5,true).',
 'entity(scale,0,6).',
 'attribute((scale,channel),6,y).',
 'attribute((scale,type),6,

In [18]:
show(
    data(['temp_max']),
    {
        'view': [
            {
                'mark': [
                    {
                        'type': 'bar',
                        'encoding': [
                            {
                                'channel': 'x',
                                'aggregate': 'mean',
                                'field': 'temp_max',
                            }
                        ],
                    },
                    {
                        'type': 'tick',
                        'encoding': [{'channel': 'x', 'field': 'temp_max'}],
                    },
                ],
                'scale': [{'channel': 'x', 'type': 'linear', 'zero': 'true'}],
            }
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "bar",
          "encoding": [
            {
              "channel": "x",
              "aggregate": "mean",
              "field": "temp_max"
            }
          ]
        },
        {
          "type": "tick",
          "encoding": [
            {
              "channel": "x",
              "field": "temp_max"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "linear",
          "zero": "true"
        }
      ]
    }
  ]
}


alt.LayerChart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(view,root,0).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,bar).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,x).',
 'attribute((encoding,aggregate),2,mean).',
 'attribute((encoding,field),2,temp_max).',
 'entity(mark,0,3).',
 'attribute((mark,type),3,tick).',
 'entity(encoding,3,4).',
 'attribute((encoding,channel),4,x).',
 'attribute((encoding,field),4,temp_max).',
 'entity(scale,0,5).',
 'attribute((scale,channel),5,x).',
 'attribute((scale,type),5,linear).',
 'attribute((scale,zero),5,true).']

In [19]:
show(
    data(['temp_max', 'wind', 'weather']),
    {
        'view': [
            {
                'mark': [
                    {
                        'type': 'point',
                        'encoding': [
                            {'channel': 'x', 'field': 'temp_max'},
                            {'channel': 'y', 'field': 'wind'},
                        ],
                    }
                ],
                'scale': [
                    {'channel': 'x', 'type': 'linear'},
                    {'channel': 'y', 'type': 'linear'},
                ],
                'facet': [
                    {'channel': 'col', 'field': 'weather'},
                ],
            }
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    },
    {
      "name": "wind",
      "type": "number",
      "__id__": "wind"
    },
    {
      "name": "weather",
      "type": "string",
      "__id__": "weather"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "point",
          "encoding": [
            {
              "channel": "x",
              "field": "temp_max"
            },
            {
              "channel": "y",
              "field": "wind"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "linear"
        },
        {
          "channel": "y",
          "type": "linear"
        }
      ],
      "facet": [
        {
          "channel": "col",
          "field": "weather"
        }
      ]
    }
  ]
}


alt.FacetChart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(field,root,wind).',
 'attribute((field,name),wind,wind).',
 'attribute((field,type),wind,number).',
 'entity(field,root,weather).',
 'attribute((field,name),weather,weather).',
 'attribute((field,type),weather,string).',
 'entity(view,root,0).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,point).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,x).',
 'attribute((encoding,field),2,temp_max).',
 'entity(encoding,1,3).',
 'attribute((encoding,channel),3,y).',
 'attribute((encoding,field),3,wind).',
 'entity(scale,0,4).',
 'attribute((scale,channel),4,x).',
 'attribute((scale,type),4,linear).',
 'entity(scale,0,5).',
 'attribute((scale,channel),5,y).',
 'attribute((scale,type),5,linear).',
 'entity(facet,0,6).',
 'attribute((facet,channel),6,col).',
 'attribute((facet,field),6,weather).']

In [20]:
show(
    data(['temp_max', 'wind', 'weather']),
    {
        'view': [
            {
                'mark': [
                    {
                        'type': 'point',
                        'encoding': [
                            {'channel': 'x', 'field': 'weather'},
                            {'channel': 'y', 'field': 'wind'},
                        ],
                    }
                ],
                'scale': [
                    {'channel': 'x', 'type': 'ordinal'},
                    {'channel': 'y', 'type': 'linear'},
                ],
                'facet': [
                    {'channel': 'col', 'field': 'temp_max', 'binning': 10},
                ],
            }
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    },
    {
      "name": "wind",
      "type": "number",
      "__id__": "wind"
    },
    {
      "name": "weather",
      "type": "string",
      "__id__": "weather"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "point",
          "encoding": [
            {
              "channel": "x",
              "field": "weather"
            },
            {
              "channel": "y",
              "field": "wind"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "ordinal"
        },
        {
          "channel": "y",
          "type": "linear"
        }
      ],
      "facet": [
        {
          "channel": "col",
          "field": "temp_max",
          "binning": 10
        }
      ]
    }
  ]
}


alt.FacetChart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(field,root,wind).',
 'attribute((field,name),wind,wind).',
 'attribute((field,type),wind,number).',
 'entity(field,root,weather).',
 'attribute((field,name),weather,weather).',
 'attribute((field,type),weather,string).',
 'entity(view,root,0).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,point).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,x).',
 'attribute((encoding,field),2,weather).',
 'entity(encoding,1,3).',
 'attribute((encoding,channel),3,y).',
 'attribute((encoding,field),3,wind).',
 'entity(scale,0,4).',
 'attribute((scale,channel),4,x).',
 'attribute((scale,type),4,ordinal).',
 'entity(scale,0,5).',
 'attribute((scale,channel),5,y).',
 'attribute((scale,type),5,linear).',
 'entity(facet,0,6).',
 'attribute((facet,channel),6,col).',
 'attribute((facet,field),6,temp_max).',
 'attribute((f

In [21]:
show(
    data(['temp_max', 'weather']),
    {
        'view': [
            {
                'mark': [
                    {
                        'type': 'tick',
                        'encoding': [{'channel': 'y', 'field': 'temp_max'}],
                    }
                ],
                'scale': [{'channel': 'y', 'type': 'linear', 'zero': 'true'}],
            },
            {
                'mark': [
                    {
                        'type': 'bar',
                        'encoding': [
                            {'channel': 'x', 'field': 'weather'},
                            {'channel': 'y', 'aggregate': 'count'},
                        ],
                    }
                ],
                'scale': [
                    {'channel': 'x', 'type': 'ordinal'},
                    {'channel': 'y', 'type': 'linear', 'zero': 'true'},
                ],
            },
        ]
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    },
    {
      "name": "weather",
      "type": "string",
      "__id__": "weather"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "tick",
          "encoding": [
            {
              "channel": "y",
              "field": "temp_max"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "y",
          "type": "linear",
          "zero": "true"
        }
      ]
    },
    {
      "mark": [
        {
          "type": "bar",
          "encoding": [
            {
              "channel": "x",
              "field": "weather"
            },
            {
              "channel": "y",
              "aggregate": "count"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "ordinal"
        },
        {
          "channel": "y",
       

alt.HConcatChart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(field,root,weather).',
 'attribute((field,name),weather,weather).',
 'attribute((field,type),weather,string).',
 'entity(view,root,0).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,tick).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,y).',
 'attribute((encoding,field),2,temp_max).',
 'entity(scale,0,3).',
 'attribute((scale,channel),3,y).',
 'attribute((scale,type),3,linear).',
 'attribute((scale,zero),3,true).',
 'entity(view,root,4).',
 'entity(mark,4,5).',
 'attribute((mark,type),5,bar).',
 'entity(encoding,5,6).',
 'attribute((encoding,channel),6,x).',
 'attribute((encoding,field),6,weather).',
 'entity(encoding,5,7).',
 'attribute((encoding,channel),7,y).',
 'attribute((encoding,aggregate),7,count).',
 'entity(scale,4,8).',
 'attribute((scale,channel),8,x).',
 'attribute((scale,type),8,ordinal

In [22]:
show(
    data(['temp_max', 'weather']),
    {
        'view': [
            {
                'mark': [
                    {
                        'type': 'tick',
                        'encoding': [{'channel': 'y', 'field': 'temp_max'}],
                    }
                ],
            },
            {
                'mark': [
                    {
                        'type': 'bar',
                        'encoding': [
                            {
                                'channel': 'y',
                                'field': 'temp_max',
                                'aggregate': 'mean',
                            },
                            {'channel': 'x', 'field': 'weather'},
                        ],
                    }
                ],
                'scale': [{'channel': 'x', 'type': 'ordinal'}],
            },
        ],
        'scale': [{'channel': 'y', 'type': 'linear', 'zero': 'true'}],
    },
)

{
  "number_rows": 1461,
  "field": [
    {
      "name": "temp_max",
      "type": "number",
      "__id__": "temp_max"
    },
    {
      "name": "weather",
      "type": "string",
      "__id__": "weather"
    }
  ],
  "view": [
    {
      "mark": [
        {
          "type": "tick",
          "encoding": [
            {
              "channel": "y",
              "field": "temp_max"
            }
          ]
        }
      ]
    },
    {
      "mark": [
        {
          "type": "bar",
          "encoding": [
            {
              "channel": "y",
              "field": "temp_max",
              "aggregate": "mean"
            },
            {
              "channel": "x",
              "field": "weather"
            }
          ]
        }
      ],
      "scale": [
        {
          "channel": "x",
          "type": "ordinal"
        }
      ]
    }
  ],
  "scale": [
    {
      "channel": "y",
      "type": "linear",
      "zero": "true"
    }
  ]
}


alt.HConcatChart(...)

['attribute(number_rows,root,1461).',
 'entity(field,root,temp_max).',
 'attribute((field,name),temp_max,temp_max).',
 'attribute((field,type),temp_max,number).',
 'entity(field,root,weather).',
 'attribute((field,name),weather,weather).',
 'attribute((field,type),weather,string).',
 'entity(view,root,0).',
 'entity(mark,0,1).',
 'attribute((mark,type),1,tick).',
 'entity(encoding,1,2).',
 'attribute((encoding,channel),2,y).',
 'attribute((encoding,field),2,temp_max).',
 'entity(view,root,3).',
 'entity(mark,3,4).',
 'attribute((mark,type),4,bar).',
 'entity(encoding,4,5).',
 'attribute((encoding,channel),5,y).',
 'attribute((encoding,field),5,temp_max).',
 'attribute((encoding,aggregate),5,mean).',
 'entity(encoding,4,6).',
 'attribute((encoding,channel),6,x).',
 'attribute((encoding,field),6,weather).',
 'entity(scale,3,7).',
 'attribute((scale,channel),7,x).',
 'attribute((scale,type),7,ordinal).',
 'entity(scale,root,8).',
 'attribute((scale,channel),8,y).',
 'attribute((scale,type